<a href="https://colab.research.google.com/github/mori8/NLP-Pytorch-practice/blob/main/Chapter_4_%EC%9E%90%EC%97%B0%EC%96%B4_%EC%B2%98%EB%A6%AC%EB%A5%BC_%EC%9C%84%ED%95%9C_%ED%94%BC%EB%93%9C_%ED%8F%AC%EC%9B%8C%EB%93%9C_%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

지난 챕터에서 가장 간단한 신경망인 퍼셉트론을 살펴봤다. 하지만 단층의 퍼셉트론으로는 데이터에서 복잡한 패턴을 학습할 수 없다는 단점이 있다. 단층 퍼셉트론의 문제점을 보여주는 대표적인 예시로는 [XOR 문제](https://wikidocs.net/24958)가 있다. 

이 챕터에서는 2가지 피드 포워드 신경망을 살펴볼 것이다. 다층 퍼셉트론(MLP)와 합성곱 신경망(CNN)이다.

## 다층 퍼셉트론(MLP)

MLP는 퍼셉트론을 구조적으로 확장한 신경망이다. 다층 퍼셉트론이라는 이름처럼 퍼셉트론을 여러 층으로 쌓아 만든 구조다. 3장에서 다룬 퍼셉트론은 입력으로 데이터 벡터를 받고 출력값 하나를 계산했지만, MLP는 많은 퍼셉트론이 모여 있으므로 **층의 출력**은 출력값 하나가 아닌 **벡터**다.

또한 층과 층 사이에 활성함수를 통한 **비선형성**을 추가할 수 있다. 비선형성을 추가해야 깊이 있는 학습이 가능해진다. 왜일까? 한 번 3개의 층을 가진 네트워크에서 $h(x) = ax$처럼 생긴 선형 활성함수를 사용했을 때를 상상해 보자. 그 결과를 살펴보면 $h(h(h(x))) = a^3x$다. 여기서 $a^3$은 상수이므로, 은닉층을 넣은 효과를 볼 수 없다. 다층 퍼셉트론에서는 꼭 비선형 활성함수를 사용해야 하는 이유다.

MLP는 은닉층을 통해 모델이 중간 표현을 학습할 수 있도록 한다. 즉 하나의 직선 혹은 (일반적으로) 초평면으로 데이터 포인트가 어느 쪽에 놓여있는지 구별할 수 있다.

#### 파이토치로 MLP 구현하기

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultilayerPerceptron(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    """
    매개변수:
      input_dim(int): 입력 벡터 크기
      hidden_dim(int): 첫번째 Linear 층의 출력 크기
      output_dim(int): 두번째 Linear 층의 출력 크기
    """
    super(MultilayerPerceptron, self).__init__()
    self.fc1 = nn.Linear(input_dim, hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, output_dim)
  
  def forward(self, x_in, apply_softmax=False):
    """MLP의 정방향 계산
    매개변수:
      x_in(torch.Tensor): 입력 데이터 센서
        x_in.shape는 (batch, input_dim)입니다.
      apply_softmax(bool): 소프트맥스 활성화 함수를 위한 플래그
        크로스 엔트로피 손실을 사용하려면 False로 지정해야 합니다.
    반환값:
      결과 텐서. tensor.shape은 (batch, output_dim)입니다.
    """
    intermediate = F.relu(self.fc1(x_in))
    output = self.fc2(intermediate)

    if apply_softmax:
      output = F.softmax(output, dim=1)
    return output

In [10]:
batch_size = 2  # 한 번에 입력할 샘플 개수
input_dim = 3
hidden_dim = 100
output_dim = 4

mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)
print(mlp)

MultilayerPerceptron(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=4, bias=True)
)


In [11]:
# 랜덤 입력으로 MLP 테스트
def describe(x):
  print("타입: {}".format(x.type()))
  print("크기: {}".format(x.shape))
  print("값: \n{}".format(x))

x_input = torch.rand(batch_size, input_dim)
describe(x_input)

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[0.8982, 0.9495, 0.5222],
        [0.9737, 0.8029, 0.2719]])


In [12]:
y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: 
tensor([[ 0.3929, -0.2453, -0.0381,  0.1673],
        [ 0.3477, -0.2822, -0.0290,  0.1500]], grad_fn=<AddmmBackward>)


파이토치 모델의 입력과 출력을 읽는 방법을 알아보자. 위 코드에서 MLP 모델의 출력은 행 2개와 열 4개가 있는 텐서다. 이 텐서의 **행**은 **배치 차원**, 즉 **미니배치의 데이터 포인트 개수**며 **열**은 각 **데이터 포인트에 대한 최종 특성 벡터**다. 분류 등에서는 특성 벡터가 곧 예측 벡터이며, 각 값이 **확률 분포에 대응**한다.

훈련을 수행하는지 추론을 수행하는지에 따라 예측 벡터로 하는 일이 다르다. 훈련 중에는 예측 벡터를 손실 함수, 타겟 클래스 레이블과 함께 사용한다. 추론에서 예측 벡터를 **확률**로 바꾸려면 `softmax` 활성함수가 필요하다. 

In [13]:
y_output = mlp(x_input, apply_softmax=True)
describe(y_output)

타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: 
tensor([[0.3360, 0.1775, 0.2184, 0.2681],
        [0.3290, 0.1752, 0.2258, 0.2700]], grad_fn=<SoftmaxBackward>)
